Average of cases divided by the number of population of each country
(TOP 10)

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder.appName('deathpercentagelocally').getOrCreate() 

covid_df = spark.read.csv("C:/Users/pooja konduri/Downloads/covid-kaggle-dataset/worldometer_data.csv", header=True, inferSchema=True)
covid_df.printSchema()

# 1. Calculate the average number of confirmed cases for each country
country_avg_cases = covid_df.groupBy("Country/Region").agg(
    F.mean("TotalCases").alias("avg_confirmed"),  # Average of confirmed cases
    F.first("Population").alias("population")  # Assuming population data is consistent for each country
)

# 2. Calculate the ratio of the average cases to population
country_avg_cases_ratio = country_avg_cases.withColumn(
    "avg_cases_per_population", F.col("avg_confirmed") / F.col("population")
)

# 3. Sort by the ratio in descending order and get the top 10 countries
top_10_avg_countries = country_avg_cases_ratio.orderBy(F.col("avg_cases_per_population").desc()).limit(10)

# Show the top 10 countries with the highest average case-to-population ratio
top_10_avg_countries.select("Country/Region", "avg_confirmed", "population", "avg_cases_per_population").show()


root
 |-- Country/Region: string (nullable = true)
 |-- Continent: string (nullable = true)
 |-- Population: integer (nullable = true)
 |-- TotalCases: integer (nullable = true)
 |-- NewCases: integer (nullable = true)
 |-- TotalDeaths: integer (nullable = true)
 |-- NewDeaths: integer (nullable = true)
 |-- TotalRecovered: integer (nullable = true)
 |-- NewRecovered: integer (nullable = true)
 |-- ActiveCases: integer (nullable = true)
 |-- Serious,Critical: integer (nullable = true)
 |-- Tot Cases/1M pop: integer (nullable = true)
 |-- Deaths/1M pop: double (nullable = true)
 |-- TotalTests: integer (nullable = true)
 |-- Tests/1M pop: integer (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------+-------------+----------+------------------------+
|Country/Region|avg_confirmed|population|avg_cases_per_population|
+--------------+-------------+----------+------------------------+
|         Qatar|     112092.0|   2807805|    0.039921575750452756|
| French Guiana| 